## Hyperdrive によるハイパーパラメータチューニング

In [ ]:
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig, Dataset
from azureml.widgets import RunDetails

In [ ]:
ws = Workspace.from_config()

### Dataset の取得

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')

### 実験名の設定

In [ ]:
experiment = Experiment(workspace=ws, name='dummy-hyperdrive2')

### 学習スクリプトの実行設定

In [ ]:

config = ScriptRunConfig(source_directory='./code/pytorch-hyperdrive',
                         script='train.py',
                         compute_target='gpucluster',
                         arguments=[
                            '--data_path', dataset.as_named_input('input').as_mount(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92])


### Environment の呼び出しと実行設定へのインプット

In [ ]:
env = Environment.get(ws, "pytorch-env")

In [ ]:
config.run_config.environment = env

### Hyperdrive におけるパラメータの設定

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice, loguniform

# パラメータ探索範囲の設定
ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-6, -1),
        '--momentum': loguniform(-6, -1),
    }
)

In [ ]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

### Hyperdrive 実行設定

In [ ]:
hyperdrive_config = HyperDriveConfig(run_config=config,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='train_loss',  # チューニング目標のメトリック
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,  # or MAXIMIZE
                                     max_total_runs=20,  # 最大試行回数
                                     max_concurrent_runs=4)  # 最大並列ど

### 実行と結果確認

In [ ]:
run = experiment.submit(hyperdrive_config)

In [ ]:
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)